First step: I install the open source machine learning package "Pycaret" developed by Smith almuni Moez Ali.

In [ ]:
!pip install --pre pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -U matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.5.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


I import pandas so I can read the CSVs into dataframes

In [ ]:
import pandas as pd
import numpy as np

I import drive so I can mount my files onto the virtual instance of the server

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


I have pandas convert my csv's into dataframes

In [ ]:
invoice_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/invoice_test.csv', low_memory=False)
invoice_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/invoice_train.csv', low_memory=False)
client_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/client_test.csv', low_memory=False)
client_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/client_train.csv', low_memory=False)
sample_submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SampleSubmission.csv', low_memory=False)

print(invoice_test.shape, invoice_train.shape, client_test.shape, client_train.shape)

(887059, 16) (864499, 16) (58069, 5) (135493, 6)


This page: https://zindi.africa/competitions/ai-hack-tunisia-4-predictive-analytics-challenge-1/data has the data dictionary. I'm using "target" as my label.

In [ ]:
client_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135493 entries, 0 to 135492
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   disrict        135493 non-null  int64  
 1   client_id      135493 non-null  object 
 2   client_catg    135493 non-null  int64  
 3   region         135493 non-null  int64  
 4   creation_date  135493 non-null  object 
 5   target         135493 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 6.2+ MB


In [ ]:
invoice_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864499 entries, 0 to 864498
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   client_id             864499 non-null  object
 1   invoice_date          864499 non-null  object
 2   tarif_type            864499 non-null  int64 
 3   counter_number        864499 non-null  int64 
 4   counter_statue        864499 non-null  int64 
 5   counter_code          864499 non-null  int64 
 6   reading_remarque      864499 non-null  int64 
 7   counter_coefficient   864499 non-null  int64 
 8   consommation_level_1  864499 non-null  int64 
 9   consommation_level_2  864499 non-null  int64 
 10  consommation_level_3  864499 non-null  int64 
 11  consommation_level_4  864499 non-null  int64 
 12  old_index             864499 non-null  int64 
 13  new_index             864499 non-null  int64 
 14  months_number         864499 non-null  int64 
 15  counter_type     

In [ ]:
def feature_change(cl, inv):

    cl['client_catg'] = cl['client_catg'].astype('category')
    cl['disrict'] = cl['disrict'].astype('category')
    cl['region'] = cl['region'].astype('category')
    cl['region_group'] = cl['region'].apply(lambda x: 100 if x<100 else 300 if x>300 else 200)
    cl['creation_date'] = pd.to_datetime(cl['creation_date'])
    
    cl['coop_time'] = (2019 - cl['creation_date'].dt.year)*12 - cl['creation_date'].dt.month

    inv['counter_type'] = inv['counter_type'].map({"ELEC":1,"GAZ":0})
    inv['counter_statue'] = inv['counter_statue'].map({0:0,1:1,2:2,3:3,4:4,5:5,769:5,'0':0,'5':5,'1':1,'4':4,'A':0,618:5,269375:5,46:5,420:5})
    
    inv['invoice_date'] = pd.to_datetime(inv['invoice_date'], dayfirst=True)
    inv['invoice_month'] = inv['invoice_date'].dt.month
    inv['invoice_year'] = inv['invoice_date'].dt.year
    inv['is_weekday'] = ((pd.DatetimeIndex(inv.invoice_date).dayofweek) // 5 == 1).astype(float)
    inv['delta_index'] = inv['new_index'] - inv['old_index']
    
    return cl, inv

In [ ]:
client_train1, invoice_train1 = feature_change(client_train, invoice_train)
client_test1, invoice_test1 = feature_change(client_test, invoice_test)

In [ ]:
def agg_feature(invoice, client_df, agg_stat):
    
    invoice['delta_time'] = invoice.sort_values(['client_id','invoice_date']).groupby('client_id')['invoice_date'].diff().dt.days.reset_index(drop=True)
    agg_trans = invoice.groupby('client_id')[agg_stat+['delta_time']].agg(['mean','std','min','max'])
    
    agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)

    df = invoice.groupby('client_id').size().reset_index(name='transactions_count')
    agg_trans = pd.merge(df, agg_trans, on='client_id', how='left')
    
    weekday_avg = invoice.groupby('client_id')[['is_weekday']].agg(['mean'])
    weekday_avg.columns = ['_'.join(col).strip() for col in weekday_avg.columns.values]
    weekday_avg.reset_index(inplace=True)
    client_df = pd.merge(client_df, weekday_avg, on='client_id', how='left')
    
    full_df = pd.merge(client_df, agg_trans, on='client_id', how='left')
    
    full_df['invoice_per_cooperation'] = full_df['transactions_count'] / full_df['coop_time']
    
    return full_df

In [ ]:
aggs = {}
aggs['consommation_level_1'] = ['mean']
aggs['consommation_level_2'] = ['mean']
aggs['consommation_level_3'] = ['mean']
aggs['consommation_level_4'] = ['mean']
agg_trans = invoice_train.groupby(['client_id', 'counter_type']).agg(aggs)
agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
agg_trans.reset_index(inplace=True)

agg_trans1 = agg_trans

aggs = {}
aggs['consommation_level_1_mean'] = ['mean']
aggs['consommation_level_2_mean'] = ['mean']
aggs['consommation_level_3_mean'] = ['mean']
aggs['consommation_level_4_mean'] = ['mean']
agg_trans = agg_trans1.groupby(['client_id']).agg(aggs)
agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
agg_trans.reset_index(inplace=True)

df = (invoice_train.groupby('client_id')
          .size()
          .reset_index(name='{}transactions_count'.format('1')))
agg_trans = pd.merge(df, agg_trans, on='client_id', how='left')

#Next, we need to merge our dataframe with client data:
#Pandas.merge function joins operations between two Data Frame objects
#test = pd.merge(client_test1,agg_trans, on='client_id', how='left')

In [ ]:
#Next, we need to group the invoice_data by the client_id column:
aggs = {}
aggs['consommation_level_1'] = ['mean']
aggs['consommation_level_2'] = ['mean']
aggs['consommation_level_3'] = ['mean']
aggs['consommation_level_4'] = ['mean']
agg_trans = invoice_test.groupby(['client_id', 'counter_type']).agg(aggs)
agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
agg_trans.reset_index(inplace=True)

agg_trans1 = agg_trans
  
aggs = {}
aggs['consommation_level_1_mean'] = ['mean']
aggs['consommation_level_2_mean'] = ['mean']
aggs['consommation_level_3_mean'] = ['mean']
aggs['consommation_level_4_mean'] = ['mean']
agg_trans = agg_trans1.groupby(['client_id']).agg(aggs)
agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
agg_trans.reset_index(inplace=True)

df = (invoice_test.groupby('client_id')
          .size()
          .reset_index(name='{}transactions_count'.format('1')))
agg_trans = pd.merge(df, agg_trans, on='client_id', how='left')

#Next, we need to merge our dataframe with client data:
#Pandas.merge function joins operations between two Data Frame objects
#test = pd.merge(client_test1,agg_trans, on='client_id', how='left')

In [ ]:
agg_stat_columns = [
 'tarif_type',
 'counter_number',
 'counter_statue',
 'counter_code',
 'reading_remarque',
 'consommation_level_1',
 'consommation_level_2',
 'consommation_level_3',
 'consommation_level_4',
 'old_index',
 'new_index',
 'months_number',
 'counter_type',
 'invoice_month',
 'invoice_year',
 'delta_index'
]

train_df1 = agg_feature(invoice_train1, client_train1, agg_stat_columns)
test_df1 = agg_feature(invoice_test1, client_test1, agg_stat_columns)

In [ ]:
train_df1 = pd.merge(train_df1,agg_trans, on='client_id', how='left')

test_df1 = pd.merge(test_df1,agg_trans, on='client_id', how='left')

print(train_df1.columns)

Index(['disrict', 'client_id', 'client_catg', 'region', 'creation_date',
       'target', 'region_group', 'coop_time', 'is_weekday_mean',
       'transactions_count', 'tarif_type_mean', 'tarif_type_std',
       'tarif_type_min', 'tarif_type_max', 'counter_number_mean',
       'counter_number_std', 'counter_number_min', 'counter_number_max',
       'counter_statue_mean', 'counter_statue_std', 'counter_statue_min',
       'counter_statue_max', 'counter_code_mean', 'counter_code_std',
       'counter_code_min', 'counter_code_max', 'reading_remarque_mean',
       'reading_remarque_std', 'reading_remarque_min', 'reading_remarque_max',
       'consommation_level_1_mean', 'consommation_level_1_std',
       'consommation_level_1_min', 'consommation_level_1_max',
       'consommation_level_2_mean', 'consommation_level_2_std',
       'consommation_level_2_min', 'consommation_level_2_max',
       'consommation_level_3_mean', 'consommation_level_3_std',
       'consommation_level_3_min', 'consomma

In [ ]:
def new_features(df):
    
    for col in agg_stat_columns:
        df[col+'_range'] = df[col+'_max'] - df[col+'_min']
        df[col+'_max_mean'] = df[col+'_max']/df[col+'_mean']
    
    return df

In [ ]:
train_df2 = new_features(train_df1)
test_df2 = new_features(test_df1)

In [ ]:
print('Initial number of columns: ', len(client_train.columns)+len(invoice_train.columns))
print('Number of columns now: ', len(train_df2.columns))

Initial number of columns:  29
Number of columns now:  116


In [ ]:
def drop(df):

    col_drop = ['client_id', 'creation_date']
    for col in col_drop:
        df.drop([col], axis=1, inplace=True)
    return df

In [ ]:
train_df = drop(train_df2)
test_df = drop(test_df2)

In [ ]:
y = train_df['target']
X = train_df.drop('target',axis=1)

feature_name = X.columns.tolist()

In [ ]:
drop_col=['reading_remarque_max','counter_statue_min','counter_type_min','counter_type_max','counter_type_range',
          'tarif_type_max', 'delta_index_min', 'consommation_level_4_mean']

X = X.drop(drop_col, axis=1)
test_df = test_df.drop(drop_col, axis=1)

In [ ]:
from pycaret.classification import *
clf=setup(data=X,target=y)

ValueError: ignored

I run 'compare_models' to have pycaret run the data through various algorithms

In [ ]:
best = compare_models()

In [ ]:
plot_model(best)

In [ ]:
evaluate_model(best)

In [ ]:
best

In [ ]:
tuned_model=tune_model(best)

In [ ]:
final_model = finalize_model(tuned_model)

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
test.insert(36, "region 199", 0)

In [ ]:
test.replace([np.inf, -np.inf], np.nan, inplace=True)
test = test.fillna(0)

In [ ]:
predictions = predict_model(final_model, data=test)
predictions

In [ ]:
submission = pd.DataFrame({
        "client_id": sample_submission["client_id"],
        "target": predictions['Label']
    })
submission.to_csv('submission.csv', index=False)